In [73]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input

In [74]:
data_dir = ('./')
class_names = ["circle", "crown", "skull", "smiley_face", "square", "star"]

In [75]:
samples_per_class = 5000

X = []
y = []

In [76]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [77]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

### Max Normalization

In [78]:
# Normalize pixel values to range 0–1 (from 0–255)
X = X.astype('float32') / 255.0
X.shape

(30000, 784)

In [79]:
# Flatten each image from 28x28 → 784 for dense layers
X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [80]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [81]:
model = Sequential([
    Input(shape=(784,), name="input_vector"),  # ✅ Updated
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(len(class_names), activation='linear')
])

In [82]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

In [83]:
model.fit(X_train,y_train, epochs=90)

Epoch 1/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.0694
Epoch 2/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5768
Epoch 3/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4831
Epoch 4/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4405
Epoch 5/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4345
Epoch 6/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4068
Epoch 7/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4073
Epoch 8/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3942
Epoch 9/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3959
Epoch 10/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3785
Epoch 11/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3909
Epoch 12/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3802
Epoch 13/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3726
Epoch 14/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3715
Epoch 15/90
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - lo

In [84]:
y_cv_pred_probs = model.predict(X_cv)   #predict probability for each label
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)  # selects the label with maximum probability

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [85]:
print(accuracy_score(y_cv,y_cv_pred))

0.9413333333333334


In [86]:
y_test_pred_probs = model.predict(X_test) 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
0.9276666666666666


In [87]:
model.save("doodle_model.keras")